In [31]:
import pandas as pd
import numpy as np

In [32]:
df = pd.read_excel('olim_2015-2024_combined.xlsx')

In [33]:
# Preprocessing

df['mikzoa'] = df['mikzoa'].fillna("לא צויין")
df = df.dropna(subset=['yeshuv_klita', 'machoz'])
# Numeric columns - ensure int64
numeric_cols = ['year_aliya', 'month_aliya', 'age', 'age_tday']
for col in numeric_cols:
    df[col] = df[col].astype('int64')

# Low cardinality categorical columns
categorical_cols = ['gender', 'machoz', 'subcont', 'erez_moza']
for col in categorical_cols:
    df[col] = df[col].astype('category')

# High cardinality columns - keep as string
string_cols = ['yeshuv_klita', 'mikzoa']
for col in string_cols:
    df[col] = df[col].astype('string')

# Update age_tday to 2025 (add 2 years)
df.loc[df['year_aliya'] < 2024, 'age_tday'] = df.loc[df['year_aliya'] < 2024, 'age_tday'] + 2




In [34]:
# Create subject column which will feature each professions's general subject

def classify_job(job):
    if pd.isna(job):
        return "לא צויין"

    job = str(job).strip()

    # 1. לא צוין / ערכים מספריים
    if job in ["לא צויין", "מקצוע לא מוגדר", "nan"] or job.isdigit():
        return "לא צויין"

    # 2. לא עבד / תלמידים / סטודנטים
    if any(word in job for word in [
        "תלמיד", "סטודנט", "ישיבה", "מקבל סעד", "מחפש עבודה", "עקרת בית"
    ]):
        return "לא עבד"

    # 3. רפואה ופרא רפואה
    if any(word in job for word in [
        "רופא", "אחות", "פסיכולוג", "פיזיוטרפ", "קרדיולוג", "רדיולוג", "שיניים",
        "מיילדת", "עובד פרה", "מטפל", "מרפא", "קלינאי", "פרמקולוג", "פתולוג",
        "רוקח", "כירופרקט", "סניטיזציה", "תזונאי", "דיאטנ"
    ]):
        return "רפואה ופרא-רפואה"

    # 4. אמנים וספורטאים
    if any(word in job for word in [
        "שחקן", "רקדן", "צייר", "פסל", "כוראוגרף", "מוסיק", "זמר", "דוגמ", "אמן",
        "במאי", "מפיק", "עיתונאי", "תסריטאי", "צילום", "ספורט", "מאמן"
    ]):
        return "אמנים וספורטאים"

    # 5. חינוך
    if any(word in job for word in [
        "מורה", "גננת", "מרצה", "חינוך", "מנהל בית ספר", "מפקח על בתי ספר"
    ]):
        return "חינוך"

    # 6. חקלאות ובעלי חיים
    if any(word in job for word in [
        "חקלא", "אגרונום", "גידול", "בעל משק", "וטרינר", "משתלה", "דייג"
    ]):
        return "חקלאות ובעלי חיים"

    # 7. חשבונאות ומשפטים
    if any(word in job for word in [
        "משפט", "עו\"ד", "עורך דין", "רואה חשבון", "חשב", "יועץ מס", "שופט",
        "נוטריון"
    ]):
        return "חשבונאות ומשפטים"

    # 8. טכנולוגיה והנדסה
    if any(word in job for word in [
        "מהנדס", "הנדסאי", "טכנאי", "אדריכל", "מפקח", "מערכות", "חשמל",
        "אלקטרוניקה", "מכונות", "כימיה", "ביוטכנולוג", "מיקרו", "אווירונאוטיקה",
        "תחבורה", "מכרות", "אנרגט", "מבנה", "מטלורג"
    ]):
        return "טכנולוגיה והנדסה"

    # 9. מדעי החיים
    if any(word in job for word in [
        "ביולוג", "גנטיק", "ביוכימ", "אימונולוג", "מיקרוביולוג", "אקולוג",
        "ביוסטטיסט", "זואולוג", "ביופיסיקאי", "מדעי החיים"
    ]):
        return "מדעי החיים"

    # 10. מדעי הרוח והחברה
    if any(word in job for word in [
        "היסטור", "פילוסופ", "סוציולוג", "פסיכולוגיה", "אנתרופולוג",
        "מדעי החברה", "יהדות", "ספרות", "בלשן", "מדעי הרוח"
    ]):
        return "מדעי הרוח והחברה"

    # 11. מדעים מדויקים
    if any(word in job for word in [
        "פיסיקאי", "פיזיקאי", "מתמטיקאי", "סטטיסטיקאי", "כימאי",
        "מדעים מדויקים", "גיאולוג", "אסטרופיסיק"
    ]):
        return "מדעים מדויקים"

    # 12. מקצועות המחשב
    if any(word in job for word in [
        "מתכנת", "תוכניתן", "מהנדס תוכנה", "מחשבים", "מפתח",
        "מערכות מידע", "מדעי מחשב", "רשת מחשב"
    ]):
        return "מקצועות המחשב"

    # 13. עובדי תעשייה, בנייה ומזון
    if any(word in job for word in [
        "איש צבא", "קצין", "שוטר", "סוהר", "מאבטח",
        "רתך", "מסגר", "עובד יצור", "קבלן", "מנופ", "זגג", "נגר",
        "בנייה", "שיפוצים", "טבח", "קונדיטור", "מזון", "חקלאות תעשייתית"
    ]):
        return "עובדי תעשייה, בנייה ומזון"

    # 14. שירותים מסחר ושיווק
    if any(word in job for word in [
        "מכירות", "איש עסקים", "שיווק", "סוכן", "קמעונאי",
        "מסעדנות", "שירותים", "מלצר", "מוכר", "פועל", "קופאי"
    ]):
        return "שרותים, מסחר,שיווק"

    # אם לא תפס כלום
    return "סווג ידנית"


# הוספת עמודת קטגוריה ל DataFrame
df["subject"] = df["mikzoa"].apply(classify_job)


In [35]:
# Classify all those who remains

manual_map = {
    "עובד סוציאלי": "מדעי הרוח והחברה",
    "רב": "שרותים, מסחר,שיווק",
    "כלכלן": "חשבונאות ומשפטים",
    "אקדמאי במדעים מדוייקים": "מדעים מדויקים",
    "בלתי מקצועי": "בלתי מקצועי",
    "בניה כללי": "עובדי תעשייה, בנייה ומזון",
    "פנסיונר / רנטייר": "לא עבד",
    "נהג מונית": "שרותים, מסחר,שיווק",
    "עובד במקצועות חופשיים": "שרותים, מסחר,שיווק",
    "מתרגם": "מדעי הרוח והחברה",
    "חרט דפוס": "עובדי תעשייה, בנייה ומזון",
    "עצמאי": "עצמאי",
    "מינהל עסקים": "חשבונאות ומשפטים",
    "יועץ השקעות": "חשבונאות ומשפטים",
    "מזכירה רפואית": "רפואה ופרא-רפואה",
    "נהג במכונית מסחרית": "שרותים, מסחר,שיווק",
    "מנהל כללי": "שרותים, מסחר,שיווק",
    "מנהל פרויקטים": "טכנולוגיה והנדסה",
    "קוסמטיקאית": "שרותים, מסחר,שיווק",
    "תופר, חייט": "עובדי תעשייה, בנייה ומזון",
    "ספר": "שרותים, מסחר,שיווק",
    "שרברב": "עובדי תעשייה, בנייה ומזון",
    "מכונאי ב.צ.נ.": "עובדי תעשייה, בנייה ומזון",
    "מדריך מקצועי": "חינוך",
    "צבע, סייד": "עובדי תעשייה, בנייה ומזון",
    "מנהל אדמיניסטרטיבי": "שרותים, מסחר,שיווק",
    "מזכירה, כתבנית, קצרנית": "שרותים, מסחר,שיווק",
    "מנהל סניף דואר": "שרותים, מסחר,שיווק",
    "פחח": "עובדי תעשייה, בנייה ומזון",
    "מנהל בית מלון ואכסניה": "שרותים, מסחר,שיווק",
    "אופטיקאי": "רפואה ופרא-רפואה",
    "שען": "עובדי תעשייה, בנייה ומזון",
    "תקשורת": "אמנים וספורטאים",
    "מנהל סניף בנק": "חשבונאות ומשפטים",
    "ספרן": "מדעי הרוח והחברה",
    "נהג אוטובוס": "שרותים, מסחר,שיווק",
    "שוער, שמש": "שרותים, מסחר,שיווק",
    "נדלן": "שרותים, מסחר,שיווק",
    "מפעיל מנוף, עגורן": "עובדי תעשייה, בנייה ומזון",
    "קרימינולוגיה קלינית": "מדעי הרוח והחברה",
    "סוחר סיטונאי": "שרותים, מסחר,שיווק",
    "פקיד ממשלתי": "שרותים, מסחר,שיווק",
    "נפח": "עובדי תעשייה, בנייה ומזון",
    "מעצב אופנה": "אמנים וספורטאים",
    "נהג משאית סמיטריילר": "שרותים, מסחר,שיווק",
    "סופר": "אמנים וספורטאים",
    "פקיד": "שרותים, מסחר,שיווק",
    "מכונאי רכב": "עובדי תעשייה, בנייה ומזון",
    "דפס, ליטוגרף": "עובדי תעשייה, בנייה ומזון",
    "סוחר קימעונאי": "שרותים, מסחר,שיווק",
    "גרפיקאי": "אמנים וספורטאים",
    "אופה": "עובדי תעשייה, בנייה ומזון",
    "פקיד משרד כללי": "שרותים, מסחר,שיווק",
    "מדריך נער חברתי": "חינוך",
    "עובד אחר ביהלומים": "עובדי תעשייה, בנייה ומזון",
    "צלם": "אמנים וספורטאים",
    "אופטימטריסט": "רפואה ופרא-רפואה",
    "מנעולן": "עובדי תעשייה, בנייה ומזון",
    "פקיד קבלת קהל": "שרותים, מסחר,שיווק",
    "צורף": "עובדי תעשייה, בנייה ומזון",
    "סנדלר": "עובדי תעשייה, בנייה ומזון",
    "תמחירן": "חשבונאות ומשפטים",
    "יועץ אירגוני": "מדעי הרוח והחברה",
    "בעלי מקצועות בתחום הארחה": "שרותים, מסחר,שיווק",
    "ביואינפורמטיקה": "מדעי החיים",
    "סופר, עתונאי": "אמנים וספורטאים",
    "ספרויות ושפות": "מדעי הרוח והחברה",
    "מרכיב מכשירים מדויקים": "עובדי תעשייה, בנייה ומזון",
    "מלונאי": "שרותים, מסחר,שיווק",
    "עובד קרמיקה": "עובדי תעשייה, בנייה ומזון",
    "שף": "עובדי תעשייה, בנייה ומזון",
    "מדעי המדינה": "מדעי הרוח והחברה",
    "פסיכוטרפיה": "רפואה ופרא-רפואה",
    "הנדסה ביורפאוית": "טכנולוגיה והנדסה",
    "פילוסוף": "מדעי הרוח והחברה",
    "הסטוריון": "מדעי הרוח והחברה",
    "מעצב גרפיקה": "אמנים וספורטאים",
    "עיסוי/מסז'יסט": "רפואה ופרא-רפואה",
    "עובד בטחון": "שרותים, מסחר,שיווק",
    "מעצב פנים": "אמנים וספורטאים",
    "עובד מעבדה כימית": "מדעים מדויקים",
    "מחסנאי": "עובדי תעשייה, בנייה ומזון",
    "ספר כלבים": "שרותים, מסחר,שיווק",
    "ריפוי בעיסוק": "רפואה ופרא-רפואה",
    "בריאות הציבור": "מדעי החיים",
    "בקרת איכות": "טכנולוגיה והנדסה",
    "עובד בזקוק סוכר ואלכוהל": "עובדי תעשייה, בנייה ומזון",
    "מרכיב מכשיר רדיו וטלויזיה": "עובדי תעשייה, בנייה ומזון",
    "בנית ציפורניים": "שרותים, מסחר,שיווק",
    "איפור מקצועי": "אמנים וספורטאים",
    "מוזג )ברמן(": "שרותים, מסחר,שיווק",
    "אבטחת איכות": "טכנולוגיה והנדסה",
    "מנהל עבודה בבנין": "עובדי תעשייה, בנייה ומזון",
    "עובדי מעבדה רפואית": "רפואה ופרא-רפואה",
    "ברוקר": "חשבונאות ומשפטים",
    "מפעיל טלפון וטלגרף": "שרותים, מסחר,שיווק",
    "קניין": "שרותים, מסחר,שיווק",
    "מערגל מתכת": "עובדי תעשייה, בנייה ומזון",
    "עמיל מכס": "שרותים, מסחר,שיווק",
    "מנהל סניף בנק ומשרד בטוח": "חשבונאות ומשפטים",
    "מנהל משרד ביטוח": "חשבונאות ומשפטים",
    "מתמטיקה ומתמטיקה שימושית": "מדעים מדויקים",
    "עובד באמנות שמושית": "אמנים וספורטאים",
    "נוטר, שומר": "שרותים, מסחר,שיווק",
    "תרפיסט": "רפואה ופרא-רפואה",
    "תפירה וטכסטיל": "עובדי תעשייה, בנייה ומזון",
    "טרפיה באומנות": "רפואה ופרא-רפואה",
    "בעל עובד במסחר סיטוני": "שרותים, מסחר,שיווק",
    "קצב ושוחט": "עובדי תעשייה, בנייה ומזון",
    "ייבוא/ייצוא": "שרותים, מסחר,שיווק",
    "עובד מטוויה,אריגה,סריגה": "עובדי תעשייה, בנייה ומזון",
    "כבאי": "עובדי תעשייה, בנייה ומזון",
    "גנן/מעצב גינות": "חקלאות ובעלי חיים",
    "אקטואר": "חשבונאות ומשפטים",
    "שמאי": "חשבונאות ומשפטים",
    "שיננית": "רפואה ופרא-רפואה",
    "מפעיל ציוד חפירה, בניה": "עובדי תעשייה, בנייה ומזון",
    "מנהל מדור ופקיד ממשלה": "שרותים, מסחר,שיווק",
    "קוואצינג/הנחיית קבוצות": "מדעי הרוח והחברה",
    "עורך סרטים": "אמנים וספורטאים",
    "בוטניקאי": "מדעי החיים",
    "יזם": "שרותים, מסחר,שיווק",
    "ארכיולוג": "מדעי הרוח והחברה",
    "בעל אולם מסיבות, מסעדה": "שרותים, מסחר,שיווק",
    "כתיבה טכנית": "אמנים וספורטאים",
    "אנטרפולוג": "מדעי הרוח והחברה",
    "שרטט": "טכנולוגיה והנדסה",
    "תולדות האמנות": "מדעי הרוח והחברה",
    "מנהל בבי\"ס על-תכון": "חינוך",
    "אנשי היטק": "מקצועות המחשב",
    "סחר בינלאומי": "שרותים, מסחר,שיווק",
    "מנהל עבודה - פחחות ורתכות": "עובדי תעשייה, בנייה ומזון",
    "שזירת פרחים": "שרותים, מסחר,שיווק",
    "גיאוגרף": "מדעי הרוח והחברה",
    "ניהול איכות": "טכנולוגיה והנדסה",
    "ארכיבר": "מדעי הרוח והחברה",
    "מרכיב מכאני": "עובדי תעשייה, בנייה ומזון",
    "כורה, חוצב": "עובדי תעשייה, בנייה ומזון",
    "מודד מוסמך": "טכנולוגיה והנדסה",
    "דייל": "שרותים, מסחר,שיווק",
    "גאופיסיקה": "מדעים מדויקים",
    "נהג קטר": "שרותים, מסחר,שיווק",
    "מנהל עיבוד יהלומים": "עובדי תעשייה, בנייה ומזון",
    "אורטופד": "רפואה ופרא-רפואה",
    "עובד נקיון במוסדות": "עובדי תעשייה, בנייה ומזון",
    "בעל עובד במסחר קמעוני": "שרותים, מסחר,שיווק",
    "מלטש יהלומים": "עובדי תעשייה, בנייה ומזון",
    "הדרולוג": "מדעים מדויקים",
    "איפור אומנותי": "אמנים וספורטאים",
    "מאבחן דידקטי": "חינוך",
    "פקיד ספריה": "מדעי הרוח והחברה",
    "מנהל טויה, אריגה, סריגה": "עובדי תעשייה, בנייה ומזון",
    "סבל": "עובדי תעשייה, בנייה ומזון",
    "טייח": "עובדי תעשייה, בנייה ומזון",
    "מתקין טלפון וטלגרף": "שרותים, מסחר,שיווק",
    "עובד בתהליך ייצור כימי": "עובדי תעשייה, בנייה ומזון",
    "מנהל משאבי-אנוש": "שרותים, מסחר,שיווק",
    "סנדלר, מייצר נעלים": "עובדי תעשייה, בנייה ומזון",
    "עובד בעבוד שבבי": "עובדי תעשייה, בנייה ומזון",
    "תדמן למלבושים, גוזר": "עובדי תעשייה, בנייה ומזון",
    "דוור": "שרותים, מסחר,שיווק",
    "מנהל בסנדלרות וייצור עור": "עובדי תעשייה, בנייה ומזון",
    "פיזיולוג": "מדעי החיים",
    "מתקין טלפון": "שרותים, מסחר,שיווק",
    "עובד במדעי טבע דומם ב.צ.נ": "מדעים מדויקים",
    "ניקוי בגדים": "עובדי תעשייה, בנייה ומזון",
    "איש לוגיסטיקה": "עובדי תעשייה, בנייה ומזון",
    "מפעיל מנועים ומשאבות": "עובדי תעשייה, בנייה ומזון",
    "עובד יעור": "חקלאות ובעלי חיים",
    "רפד": "עובדי תעשייה, בנייה ומזון",
    "עובד בנקוי יבש וצביעה": "עובדי תעשייה, בנייה ומזון",
    "עוזרת בית, מבשלת": "שרותים, מסחר,שיווק",
    "מפעיל C.N.C": "עובדי תעשייה, בנייה ומזון",
    "פיתוח תוכנה": "מקצועות המחשב",
    "גזבר": "חשבונאות ומשפטים",
    "מלחימה, מרכיבה": "עובדי תעשייה, בנייה ומזון",
    "כוון C.N.C": "עובדי תעשייה, בנייה ומזון",
    "שירותי דת": "שרותים, מסחר,שיווק",
    "כורך": "עובדי תעשייה, בנייה ומזון",
    "מתאים מתקן ציוד אלקטרוני": "עובדי תעשייה, בנייה ומזון",
    "מנהל כריה, חציבה": "עובדי תעשייה, בנייה ומזון",
    "סדר": "עובדי תעשייה, בנייה ומזון",
    "מנהל בתחום ההנדסה": "טכנולוגיה והנדסה",
    "מדען": "מדעים מדויקים",
    "כבלר, חבלר": "עובדי תעשייה, בנייה ומזון",
    "בעל מכבסה": "שרותים, מסחר,שיווק",
    "עובד זכוכית": "עובדי תעשייה, בנייה ומזון",
    "מייצר נעלים": "עובדי תעשייה, בנייה ומזון",
    "עובד בייצור מוצרי פלסטיק": "עובדי תעשייה, בנייה ומזון",
    "עובד עבוד אלקטרו-כימי": "עובדי תעשייה, בנייה ומזון",
    "צבע כלי רכב": "עובדי תעשייה, בנייה ומזון",
    "מלבין, צובע טכסטיל": "עובדי תעשייה, בנייה ומזון",
    "מנהל עבודה בבית אריזה": "עובדי תעשייה, בנייה ומזון",
    "עובד בזקוק נפט": "עובדי תעשייה, בנייה ומזון",
    "מלגזן": "עובדי תעשייה, בנייה ומזון",
    "עובד בייצור מוצרי ניר": "עובדי תעשייה, בנייה ומזון",
    "סתת": "עובדי תעשייה, בנייה ומזון",
    "טכנולוגים רפואיים": "רפואה ופרא-רפואה",
    "רוכל, בעל דוכן": "שרותים, מסחר,שיווק",
    "בוחן רכב": "עובדי תעשייה, בנייה ומזון",
    "גישור": "מדעי הרוח והחברה",
    "לימודים קלסיים": "מדעי הרוח והחברה",
    "עובד מעבדה רפואית": "רפואה ופרא-רפואה",
    "עובד במוצרי גומי": "עובדי תעשייה, בנייה ומזון",
    "תדמיתן": "עובדי תעשייה, בנייה ומזון",
    "מייצר כלי נגינה": "אמנים וספורטאים",
    "משגיח כשרות": "שרותים, מסחר,שיווק",
    "תופרת ב.צ.נ.": "עובדי תעשייה, בנייה ומזון",
    "עובד בחדר איפול": "עובדי תעשייה, בנייה ומזון",
    "ביוסטטסטיקה": "מדעי החיים",
    "עובד בייצור צמיגים": "עובדי תעשייה, בנייה ומזון",
    "בנקאי": "חשבונאות ומשפטים",
    "פקיד ב.צ.נ.": "שרותים, מסחר,שיווק",
    "פרסום": "שרותים, מסחר,שיווק",
    "מנתח פיננסי": "חשבונאות ומשפטים",
    "תועמלן רפואי": "רפואה ופרא-רפואה",
    "מפעיל אתרי תיירות": "שרותים, מסחר,שיווק",
    "ימאי": "שרותים, מסחר,שיווק",
    "יחצן": "שרותים, מסחר,שיווק",
    "עורך אירועים": "שרותים, מסחר,שיווק",
    "רפואה אלטרנטיבית": "רפואה ופרא-רפואה",
    "אמרכל": "שרותים, מסחר,שיווק",
    "אורתופטיקה": "רפואה ופרא-רפואה",
    "די'ג'יי": "אמנים וספורטאים",
    "חזאי": "מדעים מדויקים",
    "מנהל עבוד קרמיקה וזכוכית": "עובדי תעשייה, בנייה ומזון",
    "סוור, דוברן": "עובדי תעשייה, בנייה ומזון",
    "בקטריולוג": "מדעי החיים",
    "עושה כובעים": "עובדי תעשייה, בנייה ומזון",
    "דיין בבית-דין דתי": "שרותים, מסחר,שיווק",
    "תמלילן": "אמנים וספורטאים",
    "בורסקאי, מעבד פרוות": "עובדי תעשייה, בנייה ומזון",
    "מתקין בדוד": "עובדי תעשייה, בנייה ומזון",
    "קודח בארות": "עובדי תעשייה, בנייה ומזון",
    "פלדשר": "רפואה ופרא-רפואה",
    "שדרן": "אמנים וספורטאים",
    "מרכיב צנורות": "עובדי תעשייה, בנייה ומזון",
    "מחזאי": "אמנים וספורטאים",
    "בונה אניות": "עובדי תעשייה, בנייה ומזון",
    "פרוון": "עובדי תעשייה, בנייה ומזון",
    "אודיו פרוטזיסט": "רפואה ופרא-רפואה",
    "אמנות ובימוי": "אמנים וספורטאים",
    "מתקין קיטור וטורבינות": "עובדי תעשייה, בנייה ומזון",
    "אודיולוג": "רפואה ופרא-רפואה"
}
df.loc[df["subject"] == "סווג ידנית", "subject"] = \
    df.loc[df["subject"] == "סווג ידנית", "mikzoa"].map(manual_map)


In [36]:
# Create age_range column

def classify_age(age):
    try:
        age = int(age)
    except:
        return None

    if 0 <= age <= 2:
        return "גיל 0-2"
    if 3 <= age <= 5:
        return "גיל 3-5"
    if 6 <= age <= 9:
        return "גיל 6-9"
    if 10 <= age <= 14:
        return "גיל 10-14"
    if 15 <= age <= 17:
        return "גיל 15-17"
    if age == 18:
        return "גיל 18"
    if 19 <= age <= 24:
        return "גיל 19-24"
    if 25 <= age <= 34:
        return "גיל 25-34"
    if 35 <= age <= 44:
        return "גיל 35-44"
    if 45 <= age <= 49:
        return "גיל 45-49"
    if 50 <= age <= 59:
        return "גיל 50-59"
    if 60 <= age <= 64:
        return "גיל 60-64"
    if age == 65:
        return "גיל 65"
    if 66 <= age <= 74:
        return "גיל 66-74"
    if 75 <= age <= 84:
        return "גיל 75-84"
    if age >= 85:
        return "גיל 85+"

    return None

df["age_range"] = df["age"].apply(classify_age)

In [37]:
df['subcont'] = df['subcont'].astype(str)
df.loc[df['erez_moza'] == 'תורכיה', 'subcont'] = 'אסיה'

# Function to classify the continent
def get_continent(val):
    val = str(val).strip()

    # Exact matches
    if val == 'אוקיאניה': return 'אוקיאניה'
    if val == 'צפון אמריקה': return 'צפון אמריקה'
    if val == 'דרום אמריקה': return 'דרום אמריקה'

    # "Contains" logic
    if 'אפריקה' in val: return 'אפריקה'
    if 'אסיה' in val: return 'אסיה'
    if 'אירופה' in val: return 'אירופה'

    return val # Fallback

# 3. Insert the 'continent' column right before 'subcont'
# We check if it already exists to avoid errors if you run the cell twice
if 'continent' not in df.columns:
    idx = df.columns.get_loc('subcont')
    df.insert(idx, 'continent', df['subcont'].apply(get_continent))
else:
    # If it exists, just update the values
    df['continent'] = df['subcont'].apply(get_continent)

# Display to verify
df[df['erez_moza'] == 'תורכיה'][['erez_moza', 'subcont', 'continent']].head()

,erez_moza,subcont,continent
327,תורכיה,אסיה,אסיה
830,תורכיה,אסיה,אסיה
831,תורכיה,אסיה,אסיה
960,תורכיה,אסיה,אסיה
961,תורכיה,אסיה,אסיה


In [38]:

col = 'erez_moza'

df[col] = df[col].str.strip()

vals_to_drop = [
    "שאר אמריקה המרכזית",
    "אפריקה ב.צ.נ.",
    "שאר אמריקה הצפונית",
    "שאר אמריקה הדרומית",
    "שאר אפריקה",
    "שאר אירופה",
    "שאר אוקיאניה"
]

df = df[~df[col].isin(vals_to_drop)]

replacements = {
    "זימבבואה (רודזיה)": "זימבבואה",
    "זימבבוה": "זימבבואה",
    "קוריאה": "קוריאה הדרומית",
    'בריה"מ': "רוסיה",
    "יוגוסלביה": "בוסניה הרצגובינה",
    "ויטנאם הדרומית": "ויאטנם",
    "ויטנאם הצפונית": "ויאטנם",
    "ויטנאם": "ויאטנם",
    "איחוד האמירויות הער": "איחוד האמירויות",
    "אל סלואדור": "אל סלבדור",
    "גמייקה": "ג'מייקה",
    "הרפ הדומיניקנית": "הרפובליקה הדומיניקנית",
    "טיוון": "טאיוואן",
    "טנזאניה": "טנזניה",
    "מולדביה": "מולדובה",
    "מכסיקו": "מקסיקו",
    "סט וינסנט": "סנט וינסנט",
    "סרי לנקה (צילון)": "סרי לנקה",
    "עירק": "עיראק",
    "צ'ילי": "צ'ילה",
    "קטר": "קטאר",
    "תורכיה": "טורקיה",
    "אירן": "איראן"


}

df[col] = df[col].replace(replacements)

In [39]:
# 1. Dictionary of manual corrections: {Country Name: Correct Continent}
country_corrections = {
    "איחוד האמירויות": "אסיה",
    "קטאר": "אסיה",
    "איי סיישל": "אפריקה",      # Note: Make sure the spacing matches your data exactly
    "איי  סיישל": "אפריקה",     # Added version with double space just in case
    "סנט וינסנט": "דרום אמריקה",
    "קלדוניה החדשה": "אוקיאניה"
}

# 2. Apply the corrections
for country, new_continent in country_corrections.items():
    # Update the 'continent' column where 'erez_moza' matches the country
    df.loc[df['erez_moza'].str.strip() == country, 'continent'] = new_continent

# 3. Verify the changes (display specific rows to check)
check_countries = list(country_corrections.keys())
df[df['erez_moza'].isin(check_countries)][['erez_moza', 'continent']].drop_duplicates()

,erez_moza,continent
24440,קלדוניה החדשה,אוקיאניה
59671,סנט וינסנט,דרום אמריקה
66278,איי סיישל,אפריקה
133402,קטאר,אסיה
229664,איחוד האמירויות,אסיה


In [40]:
# 1. Ensure 'subcont' is a string so we can write new values into it
df['subcont'] = df['subcont'].astype(str)

# 2. Find rows where 'subcont' is "אחרות"
mask = df['subcont'].str.strip() == 'אחרות'

# 3. For those rows, overwrite 'subcont' with the value from 'continent'
df.loc[mask, 'subcont'] = df.loc[mask, 'continent']

# 4. Verify: Check if 'אחרות' is gone from 'subcont'
# (It will be gone if every 'Other' country got a continent assigned)
print("Remaining 'Other' in subcont:", df[df['subcont'] == 'אחרות'].shape[0])
df.head()

Remaining 'Other' in subcont: 0


,year_aliya,month_aliya,age,age_tday,gender,erez_moza,continent,subcont,yeshuv_klita,machoz,mikzoa,subject,age_range
0,2015,10,18,28,נקבה,"ארה""ב",צפון אמריקה,צפון אמריקה,שחר,מחוז הדרום וירושלים,לא צויין,לא צויין,גיל 18
1,2015,7,38,48,נקבה,בריטניה,אירופה,מערב אירופה,חצבה,מחוז הדרום וירושלים,גננת,חינוך,גיל 35-44
2,2015,3,35,45,זכר,אוקראינה,אירופה,מזרח אירופה,מזכרת בתיה,מחוז תל אביב והמרכז,עובד סוציאלי,מדעי הרוח והחברה,גיל 35-44
3,2015,3,32,43,נקבה,אוקראינה,אירופה,מזרח אירופה,מזכרת בתיה,מחוז תל אביב והמרכז,שיווק ויחסי ציבור,"שרותים, מסחר,שיווק",גיל 25-34
4,2015,3,3,14,נקבה,אוקראינה,אירופה,מזרח אירופה,מזכרת בתיה,מחוז תל אביב והמרכז,לא צויין,לא צויין,גיל 3-5


In [41]:
df.head(20)

,year_aliya,month_aliya,age,age_tday,gender,erez_moza,continent,subcont,yeshuv_klita,machoz,mikzoa,subject,age_range
0,2015,10,18,28,נקבה,"ארה""ב",צפון אמריקה,צפון אמריקה,שחר,מחוז הדרום וירושלים,לא צויין,לא צויין,גיל 18
1,2015,7,38,48,נקבה,בריטניה,אירופה,מערב אירופה,חצבה,מחוז הדרום וירושלים,גננת,חינוך,גיל 35-44
2,2015,3,35,45,זכר,אוקראינה,אירופה,מזרח אירופה,מזכרת בתיה,מחוז תל אביב והמרכז,עובד סוציאלי,מדעי הרוח והחברה,גיל 35-44
3,2015,3,32,43,נקבה,אוקראינה,אירופה,מזרח אירופה,מזכרת בתיה,מחוז תל אביב והמרכז,שיווק ויחסי ציבור,"שרותים, מסחר,שיווק",גיל 25-34
4,2015,3,3,14,נקבה,אוקראינה,אירופה,מזרח אירופה,מזכרת בתיה,מחוז תל אביב והמרכז,לא צויין,לא צויין,גיל 3-5
5,2015,7,37,47,זכר,קנדה,צפון אמריקה,צפון אמריקה,מזכרת בתיה,מחוז תל אביב והמרכז,רופא,רפואה ופרא-רפואה,גיל 35-44
6,2015,7,5,15,נקבה,קנדה,צפון אמריקה,צפון אמריקה,מזכרת בתיה,מחוז תל אביב והמרכז,לא צויין,לא צויין,גיל 3-5
7,2015,1,57,68,זכר,"ארה""ב",צפון אמריקה,צפון אמריקה,יסוד המעלה,מחוז הצפון,פיסיקאי,מדעים מדויקים,גיל 50-59
8,2015,9,19,29,נקבה,מקסיקו,צפון אמריקה,צפון אמריקה,אופקים,מחוז הדרום וירושלים,סטודנט,לא עבד,גיל 19-24
9,2015,2,41,52,זכר,אוקראינה,אירופה,מזרח אירופה,אופקים,מחוז הדרום וירושלים,מתכנת,מקצועות המחשב,גיל 35-44


In [42]:
df.tail(10)

,year_aliya,month_aliya,age,age_tday,gender,erez_moza,continent,subcont,yeshuv_klita,machoz,mikzoa,subject,age_range
362255,2024,12,18,19,נקבה,"ארה""ב",צפון אמריקה,צפון אמריקה,באר שבע,מחוז הדרום וירושלים,עובד במקצועות חופשיים,"שרותים, מסחר,שיווק",גיל 18
362256,2024,12,32,33,זכר,רוסיה,אירופה,מזרח אירופה,באר שבע,מחוז הדרום וירושלים,מורה למדעי החברה,חינוך,גיל 25-34
362257,2024,12,22,23,זכר,צרפת,אירופה,מערב אירופה,באר שבע,מחוז הדרום וירושלים,מכונאי רכב,"עובדי תעשייה, בנייה ומזון",גיל 19-24
362258,2024,12,20,21,זכר,"ארה""ב",צפון אמריקה,צפון אמריקה,באר שבע,מחוז הדרום וירושלים,מאמן כדורסל/כדורגל,אמנים וספורטאים,גיל 19-24
362259,2024,12,44,45,זכר,מרטניק,דרום אמריקה,דרום אמריקה,אשדוד,מחוז הדרום וירושלים,עצמאי,עצמאי,גיל 35-44
362260,2024,12,18,19,זכר,אוקראינה,אירופה,מזרח אירופה,אשדוד,מחוז הדרום וירושלים,מקצוע לא מוגדר,לא צויין,גיל 18
362261,2024,12,69,70,נקבה,צרפת,אירופה,מערב אירופה,אשדוד,מחוז הדרום וירושלים,פנסיונר / רנטייר,לא עבד,גיל 66-74
362262,2024,12,44,45,זכר,צרפת,אירופה,מערב אירופה,אשדוד,מחוז הדרום וירושלים,מנהל כללי,"שרותים, מסחר,שיווק",גיל 35-44
362263,2024,12,69,70,זכר,אוסטרליה,אוקיאניה,אוקיאניה,גבעת שמואל,מחוז תל אביב והמרכז,מהנדס אזרחי,טכנולוגיה והנדסה,גיל 66-74
362264,2024,12,67,68,נקבה,אוסטרליה,אוקיאניה,אוקיאניה,גבעת שמואל,מחוז תל אביב והמרכז,מורה במוסד להשכלה גבוהה,חינוך,גיל 66-74


In [43]:

from google.colab import files

# Save as CSV (faster than Excel)
df.to_csv('olim_2015-2024_preprocessed.csv', index=False, encoding='utf-8-sig')

# Download it
files.download('olim_2015-2024_preprocessed.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>